# Image Detector from picture

This script detect objects from a picture. By default script is set for a 1280 x 1024 image.
ML model is yolo4.  It was trained with 320x320 images to detect the following objects:

 ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

When image is shown there will be in the window as buttons as objects we have pre-selected in the code to be identified. When the button is active the bouding borders are draw in the image, otherwise the object is not bounding bordered.

The image showing stops when the ESC key is pressed. 

The output image is saved into "/image_output" folder and names "image_output.png"

## Import modules

In [84]:
import cv2
from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename
from gui_buttons import Buttons
from numpy import append


## Create a button for classes to be detected

In [85]:
button = Buttons()
button.add_button("person",5,5)
button.add_button("car",5,25)
button.add_button("cell phone",5,45)
button.add_button("bicycle",5,65)
button.add_button("sports ball",5,85)

colors = button.colors

## AI Model Parameters

In [86]:
net = cv2.dnn.readNet('dnn_model\yolov4-tiny.weights','dnn_model\yolov4-tiny.cfg')
model = cv2.dnn_DetectionModel(net)
size = (320,320)
model.setInputParams(size=size, scale=1/255)


## Function definition for mouse clicks on buttons

In [87]:
def click_button(event, x,y,flags,params):
    global button_object
    if event == cv2.EVENT_LBUTTONDOWN:
        button.button_click(x,y)
        

## Window creation to be selected to the mouse callback mode 

In [88]:
cv2.namedWindow('Frame')
cv2.setMouseCallback('Frame',click_button)

## Read the classes file and generate a numpy array with them

In [89]:
classes=[]
with open('dnn_model\classes.txt','r') as file_object:
    for class_name in file_object.readlines():
        class_name = class_name.strip()
        classes.append(class_name)

## Read the image 

In [90]:


Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
img = askopenfilename() # show an "Open" dialog box and return the path to the selected file
src = cv2.imread(img, cv2.IMREAD_UNCHANGED)

## Resize to a convinient image size (the larger the size, the better the detection and worst time processing lap)

In [92]:
#percent by which the image is resized
scale_percent = size[0]/src.shape[0]

# dsize
dsize = ( int(src.shape[1]*scale_percent), size[0])

# resize image
frame = cv2.resize(src, dsize)


## Read the resized image and run the model to predict. Draw the bounding borders and classes for objects according selected buttons. Shows the image with bboxes, until a key is pressed, then save the image. 

In [93]:


while True:
    frame = cv2.resize(src, dsize)

    active_buttons=button.active_buttons_list()

    (class_ids, scores, bboxes) = model.detect(frame,confThreshold=0.3, nmsThreshold=0.4)

    for class_id, score, bbox in zip(class_ids, scores, bboxes):
        (x, y, w , h) = bbox
        class_name = classes[class_id]
        color = colors[class_id]

        if class_name in active_buttons:
            cv2.putText(frame,str(class_name), (x,y-10), cv2.FONT_HERSHEY_PLAIN,2, color,2 )
            cv2.rectangle(frame,(x,y),(x+w,y+h),color,3)

    
    button.display_buttons(frame)
    cv2.imshow("Frame",frame)
    
    key =  cv2.waitKey(1)
    if key == 27    :
        break

cv2.imwrite("//image_output//image_output.png",frame)
cv2.destroyAllWindows() 

In [94]:
cv2.destroyAllWindows() 